# Introduction

In this Juptyer Notebook file, we will introduce the _separability criteria_ , specifically the DPS hierarchy and the DPS\* criterion, stored in `qittoolbox.qit` . 

In [1]:
from qittoolbox.qit import separability as sep

import numpy as np
from scipy import linalg
from qittoolbox.cqgchannels import BC16channel, SNpluschannel
from qittoolbox.qfunctions import qfunctions
from qittoolbox.linalg import basis_transformation
from qittoolbox.linalg.tensors import tens
from math import sqrt
from qittoolbox.logging import logger

logger.set_loglevel('info')

[info] : [set_loglevel] : set loglevel to `info`


## Case study: $(k,l,m) = (1,1,2)$, quantum channels $\Phi_k^{(l),m}$ and $\hat{\Phi}_k^{(l),m}$

We first calculate the Choi matrices of the corresponding quantum channels, using the `get_choi_matrix` and `get_choi_matrix_NC` functions from the channel implementations. 

Then, we reduce the size of these Choi matrices by, also see [the tutorial on $O_N^+$ and $S_N^+$ channels](tutorial_ONplus_and_SNplus_channels.ipynb) , using the fact that the quantum channels are only defined on a _subspace_ of the tensor product spaces $\mathcal{H}^{\otimes n}$ for some $n \in \mathbb{N}$. 

Afterwards, let us store this data in the `choi_mat_data` dictionary, so that we can access it later to run the DPS and DPS\* criterion.

In [2]:
N = 4
m = 1
k = 1
l = k + m 
trace_out_first = True
threshold = 1e-10

choi_mat_data = {}
channel_names = ['ONplus', 'SNplus']
jones_wenzl_f_dict = {'ONplus': BC16channel.get_Jones_Wenzl_projection , 'SNplus': SNpluschannel.get_Jones_Wenzl_projection_NC}
choi_mat_f_dict = {'ONplus': BC16channel.get_choi_matrix , 'SNplus': SNpluschannel.get_choi_matrix_NC}


has_error = False

dim_in_full = N**k
dim_out_full = N**m if trace_out_first else N**l

for channel_name in channel_names:
    dim_pk = qfunctions.q0_bracket(k+1,N=N) if channel_name=='ONplus' else qfunctions.q0_bracket(2*k+1,N=sqrt(N))
    dim_pm = qfunctions.q0_bracket(m+1,N=N) if channel_name=='ONplus' else qfunctions.q0_bracket(2*m+1,N=sqrt(N))
    dim_pl = qfunctions.q0_bracket(l+1,N=N) if channel_name=='ONplus' else qfunctions.q0_bracket(2*l+1,N=sqrt(N))
    pk = jones_wenzl_f_dict[channel_name](k,N=N)
    pm = jones_wenzl_f_dict[channel_name](m,N=N)
    pl = jones_wenzl_f_dict[channel_name](l,N=N)
        
    print(f'====> Getting data for parameters channel={channel_name},N={N}, trace_out_first={trace_out_first}, k={k},m={m}.')
            
    choi_mat, handedness = choi_mat_f_dict[channel_name](k,l,m,N=N,trace_out_first=trace_out_first)
    choi_mat /= dim_pk

    #Check whether the normalized choi mat is a state
    if abs( np.trace(choi_mat.A) - 1 ) > threshold:
        print(f'Error! Tr(choi_mat) = {np.trace(choi_mat.A)}, should be 1.')
        has_error = True
    if any( x < -threshold for x in linalg.eigvalsh(choi_mat.A) ):
        print(f'Error! An eigenvalue was smaller than -{threshold:.1e} : \n{linalg.eigvalsh(choi_mat.A)} ')
        has_error = True

    #Make the Choi mat smaller by using partial isometries
    rank_in = dim_pk
    rank_out = dim_pm if trace_out_first else dim_pl
    p_out = pm if trace_out_first else pl
    isom_in = basis_transformation.get_isometry_from_range_space_proj(pk, rank_in)
    isom_out = basis_transformation.get_isometry_from_range_space_proj(p_out, rank_out)

    # Use the fact that the handedness tells us where the output H_m Hilbert space is located.
    full_isom = tens(isom_out, isom_in) if handedness == 'left' else tens(isom_in, isom_out)
    choi_small = full_isom.T @ choi_mat @ full_isom

    #This smaller choi matrix, although generally not sparse anymore, should still be a state
    if abs( np.trace(choi_small.A) - 1 ) > threshold:
        print(f'Error! Tr(choi_small) = {np.trace(choi_small.A)}, should be 1.')
        has_error = True
    if any( x < -threshold for x in linalg.eigvalsh(choi_small.A) ):
        print(f'Error! An eigenvalue was smaller than -{threshold:.1e} : \n{linalg.eigvalsh(choi_small.A)} ')
        has_error = True

    if not has_error:
        #Prepare the dimensionality of the Choi matrix for the PPT / k-symmetric (inner) extensions.
        dim_A = dim_out_full if handedness == 'left' else dim_in_full
        dim_B = dim_in_full if handedness == 'left' else dim_out_full

        dim_A_small = dim_pm if handedness == 'left' else dim_pk
        dim_B_small = dim_pk if handedness == 'left' else dim_pm

        fname = f'chan_{channel_name}_phi_k{k}_l{l}_m{m}_first{trace_out_first}_N{N}_data.mat'
        # data = {'choi_mat': choi_mat.A, 'dim_A':dim_A, 'dim_B':dim_B, 'choi_mat_small': choi_small.A, 'dim_A_small': dim_A_small, 'dim_B_small': dim_B_small}
        # savemat(fname,data)
        choi_mat_data[fname] = {'choi_mat': choi_mat.A, 'dim_A':dim_A, 'dim_B':dim_B, 'choi_mat_small': choi_small.A, 'dim_A_small': dim_A_small, 'dim_B_small': dim_B_small}
        print(f'>> Successfully stored data in `{fname}`')

if has_error:
    print('Finished with errors...')
else:
    print('=======================> Finished successfully!')
        

====> Getting data for parameters channel=ONplus,N=4, trace_out_first=True, k=1,m=1.
[warning] : [get_T_intertwiner] r == 0, returning [1x1] matrix. This might lead to untested behaviour.
[warning] : [get_isometry_from_range_space_proj] Warning, rank = 4 = proj.shape[0], so we are returning sparse eye(rank). Check whether this is what you want!
[warning] : [get_isometry_from_range_space_proj] Warning, rank = 4 = proj.shape[0], so we are returning sparse eye(rank). Check whether this is what you want!
>> Successfully stored data in `chan_ONplus_phi_k1_l2_m1_firstTrue_N4_data.mat`
====> Getting data for parameters channel=SNplus,N=4, trace_out_first=True, k=1,m=1.
>> Successfully stored data in `chan_SNplus_phi_k1_l2_m1_firstTrue_N4_data.mat`
=======================> Finished successfully!


Having constructed the small Choi matrices of the quantum channels from this case study, we can use `sep.symmetric_inner_extension` for the DPS\* criterion and `sep.symmetric_extension` for the DPS hierarchy to determine whether the Choi matrices are separable. Note that the test in the first case is inconclusive, but in the seoncd case we can determine that the Choi matrix of the $S_N^+$-channel is actually negative-partial transpose, and is therefore not separable!

In [3]:
logger.set_loglevel('debug')

USE_SMALLER_CHOI = True
USE_PTT_CRITERION = True
MAX_K_EXT = 2

def run_sep(fname: str, start_k_ext: int=2) -> None:
    global threshold, USE_SMALLER_CHOI, USE_PTT_CRITERION, MAX_K_EXT, choi_mat_data

    data_dict = choi_mat_data[fname]
    choi_mat = data_dict['choi_mat']
    dim_A = data_dict['dim_A']
    dim_B = data_dict['dim_B']
    choi_small = data_dict['choi_mat_small']
    dim_A_small = data_dict['dim_A_small']
    dim_B_small = data_dict['dim_B_small']

    #First, check whether the Choi matrix is PPT
    logger.log(f'>> Checking the PPT criterion for the Choi matrix...','info')
    choi_mat_ppt = np.transpose( choi_mat.reshape((dim_A,dim_B,dim_A,dim_B)), axes=[0,3,2,1] ).reshape((dim_A*dim_B,dim_A*dim_B))
    eigvals = linalg.eigvalsh(choi_mat_ppt)
    if any(x < -threshold for x in eigvals):
        logger.log(f'This Choi matrix is NPT, with most negative eigval {min(eigvals)}. Therefore, it is not separable!','info')
        return
    else:
        logger.log('Choi matrix is PPT, so continuing...','info')

    #Otherwise, run the symmetric extensions
    is_separable = False
    k_ext = start_k_ext
    while k_ext <= MAX_K_EXT:
        if USE_SMALLER_CHOI:
            prob_inner = sep.symmetric_inner_extension(choi_small,dim_A_small,dim_B_small,k=k_ext,ppt=USE_PTT_CRITERION, interface='picos' )
        else:
            prob_inner = sep.symmetric_inner_extension(choi_mat,dim_A,dim_B,k=k_ext,ppt=USE_PTT_CRITERION, interface='picos')

        if prob_inner.status == 'optimal':
            logger.log('Output status is optimal! Check whether this means that the state is separable...','info')
            is_separable = True
            break
        else:
            logger.log(f'Output status is infeasible, so this Choi matrix is {k_ext}-inner sym extensible.','info')

        if USE_SMALLER_CHOI:
            prob_outer = sep.symmetric_extension(choi_small,dim_A_small,dim_B_small,k=k_ext,ppt=USE_PTT_CRITERION, interface='picos')
        else:
            prob_outer = sep.symmetric_extension(choi_mat,dim_A,dim_B,k=k_ext,ppt=USE_PTT_CRITERION, interface='picos')

        if prob_outer.status != 'optimal':
            logger.log('Output status is not optimal! Check whether this means that the state is separable...','info')
            is_separable = True
            break
        else:
            logger.log(f'Output status is optimal, so this Choi matrix is {k_ext}-sym extensible.','info')

        k_ext += 1

    if not is_separable:
        logger.log('Could not determine whether this Choi matrix is separable...','info')


for fname in choi_mat_data.keys():
    print(f'=================> Found the data {fname}...')

    run_sep(fname)



[info] : [set_loglevel] : set loglevel to `debug`
=================> Found the data chan_ONplus_phi_k1_l2_m1_firstTrue_N4_data.mat...
[info] : >> Checking the PPT criterion for the Choi matrix...
[info] : Choi matrix is PPT, so continuing...
[info] : [symmetric_extension] ========================= [STARTING 2-INNER-SYMMETRIC EXTENSION] ====================
[info] : [symmetric_inner_extension] Computed dimensionalities as: 
dim_A = 4, dim_B = 4, extend_B=False, dim_symmetric_out = 10, k = 2, total_dim_out = 40
[info] : [symmetric_inner_extension] Computing the permutation operator... For large dimensions this may take some time.
[info] : [symmetric_inner_extension] Computed the permutation operator in 3.00e-03s.
[info] : [symmetric_inner_extension] Current problem: 
Complex Feasibility Problem
  find an assignment
  for
    40×40 hermitian variable rho_extended
  subject to
    rho_extended ≽ 0
    tr(rho_extended) = 1
    0.392202·([64×40]·rho_extended·[40×64]).{tr([4×4])⊗[4×4]⊗[4×4]} 